# Weather ML Prediction Analysis

## Objective
Analyze model predictions to understand:
- Performance by city
- Performance by hour of day
- Performance by day of week
- Performance by weather condition
- Prediction error distributions
- Residual patterns

This notebook:
1. Loads predictions from `weather_predictions` table
2. Performs city-wise analysis
3. Performs hourly analysis
4. Performs day-of-week analysis
5. Performs condition-based analysis
6. Saves analysis results to `weather_analysis` table

**Input Table:** `weather_predictions`
**Output Table:** `weather_analysis`

In [ ]:
# Import Required Libraries
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import (
    col, abs as spark_abs, round as spark_round, 
    avg, stddev, max as spark_max, min as spark_min,
    count, when, desc
)
from pyspark.sql.types import DoubleType
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✓ Libraries imported successfully")

## Section 1: Load Predictions

Load the model predictions from `weather_predictions` table

In [ ]:
# Load predictions
try:
    predictions = spark.table("weather_predictions")
    logger.info("✓ Loaded weather_predictions table")
except Exception as e:
    logger.error(f"Failed to load table: {e}")
    raise

print(f"Loaded Predictions:")
print(f"  Total records: {predictions.count():,}")
print(f"  Columns: {len(predictions.columns)}")

# Ensure abs_error column exists
if "abs_error" not in predictions.columns:
    predictions = predictions.withColumn(
        "abs_error",
        spark_abs(col("prediction") - col("temperature")).cast(DoubleType())
    )
    print("  Created abs_error column")

# Display sample
print("\nSample Predictions:")
predictions.select("city", "timestamp", "temperature", "prediction", "abs_error").show(5)

## Section 2: City-Wise Performance Analysis

Analyze prediction accuracy by city

In [ ]:
# City-wise performance analysis
print("\n" + "=" * 80)
print("CITY-WISE PERFORMANCE ANALYSIS")
print("=" * 80)

city_analysis = predictions.groupBy("city").agg(
    count("*").alias("predictions"),
    spark_round(avg("abs_error"), 4).alias("avg_error"),
    spark_round(stddev("abs_error"), 4).alias("std_error"),
    spark_round(spark_min("abs_error"), 4).alias("min_error"),
    spark_round(spark_max("abs_error"), 4).alias("max_error"),
    spark_round(avg("temperature"), 2).alias("avg_actual_temp"),
    spark_round(avg("prediction"), 2).alias("avg_predicted_temp"),
    (count(when(col("abs_error") <= 1.0, 1)) / count("*") * 100).alias("accuracy_within_1c_pct"),
    (count(when(col("abs_error") <= 2.0, 1)) / count("*") * 100).alias("accuracy_within_2c_pct")
).orderBy("avg_error")

city_analysis.show(truncate=False)

# Save city analysis
city_analysis.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("weather_analysis_by_city")

## Section 3: Hourly Performance Analysis

Analyze prediction accuracy by hour of day

In [ ]:
# Hourly performance analysis (if hour column exists)
if "hour" in predictions.columns:
    print("\n" + "=" * 80)
    print("HOURLY PERFORMANCE ANALYSIS")
    print("=" * 80)
    
    hourly_analysis = predictions.groupBy("hour").agg(
        count("*").alias("predictions"),
        spark_round(avg("abs_error"), 4).alias("avg_error"),
        spark_round(stddev("abs_error"), 4).alias("std_error"),
        spark_round(avg("temperature"), 2).alias("avg_actual_temp"),
        spark_round(avg("prediction"), 2).alias("avg_predicted_temp"),
        (count(when(col("abs_error") <= 2.0, 1)) / count("*") * 100).alias("accuracy_within_2c_pct")
    ).orderBy("hour")
    
    hourly_analysis.show(truncate=False)
    
    # Save hourly analysis
    hourly_analysis.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("weather_analysis_by_hour")
else:
    print("Hour column not available for hourly analysis")

## Section 4: Day-of-Week Performance Analysis

Analyze prediction accuracy by day of week

In [ ]:
# Day-of-week performance analysis
if "day_of_week" in predictions.columns:
    print("\n" + "=" * 80)
    print("DAY-OF-WEEK PERFORMANCE ANALYSIS")
    print("=" * 80)
    
    day_names = {1: "Sunday", 2: "Monday", 3: "Tuesday", 4: "Wednesday",
                 5: "Thursday", 6: "Friday", 7: "Saturday"}
    
    daily_analysis = predictions.groupBy("day_of_week").agg(
        count("*").alias("predictions"),
        spark_round(avg("abs_error"), 4).alias("avg_error"),
        spark_round(stddev("abs_error"), 4).alias("std_error"),
        spark_round(avg("temperature"), 2).alias("avg_actual_temp"),
        (count(when(col("abs_error") <= 2.0, 1)) / count("*") * 100).alias("accuracy_within_2c_pct")
    ).withColumn(
        "day_name",
        when(col("day_of_week") == 1, "Sunday")
        .when(col("day_of_week") == 2, "Monday")
        .when(col("day_of_week") == 3, "Tuesday")
        .when(col("day_of_week") == 4, "Wednesday")
        .when(col("day_of_week") == 5, "Thursday")
        .when(col("day_of_week") == 6, "Friday")
        .when(col("day_of_week") == 7, "Saturday")
    ).orderBy("day_of_week")
    
    daily_analysis.show(truncate=False)
    
    # Save daily analysis
    daily_analysis.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("weather_analysis_by_day")

## Section 5: Weather Condition Performance Analysis

Analyze prediction accuracy by weather condition

In [ ]:
# Weather condition performance analysis
if "condition" in predictions.columns:
    print("\n" + "=" * 80)
    print("WEATHER CONDITION PERFORMANCE ANALYSIS")
    print("=" * 80)
    
    condition_analysis = predictions.groupBy("condition").agg(
        count("*").alias("predictions"),
        spark_round(avg("abs_error"), 4).alias("avg_error"),
        spark_round(stddev("abs_error"), 4).alias("std_error"),
        spark_round(spark_min("abs_error"), 4).alias("min_error"),
        spark_round(spark_max("abs_error"), 4).alias("max_error"),
        spark_round(avg("temperature"), 2).alias("avg_actual_temp"),
        (count(when(col("abs_error") <= 2.0, 1)) / count("*") * 100).alias("accuracy_within_2c_pct")
    ).orderBy("avg_error")
    
    condition_analysis.show(truncate=False)
    
    # Save condition analysis
    condition_analysis.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("weather_analysis_by_condition")

## Section 6: Residual and Error Distribution Analysis

Analyze error distribution and residuals

In [ ]:
# Residual analysis
print("\n" + "=" * 80)
print("RESIDUAL AND ERROR DISTRIBUTION ANALYSIS")
print("=" * 80)

residual_stats = predictions.select(
    spark_round(avg("abs_error"), 4).alias("mean_abs_error"),
    spark_round(stddev("abs_error"), 4).alias("std_abs_error"),
    spark_round(spark_min("abs_error"), 4).alias("min_abs_error"),
    spark_round(spark_max("abs_error"), 4).alias("max_abs_error")
).collect()[0]

print("\nError Statistics:")
print(f"  Mean Absolute Error:     {residual_stats['mean_abs_error']:.4f}°C")
print(f"  Std Dev Absolute Error:  {residual_stats['std_abs_error']:.4f}°C")
print(f"  Min Absolute Error:      {residual_stats['min_abs_error']:.4f}°C")
print(f"  Max Absolute Error:      {residual_stats['max_abs_error']:.4f}°C")

# Accuracy by error range
print("\nPrediction Accuracy by Error Range:")
for error_threshold in [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]:
    accuracy = predictions.filter(col("abs_error") <= error_threshold).count() / predictions.count() * 100
    print(f"  Within {error_threshold}°C: {accuracy:.1f}% of predictions")

# Save residual statistics
residual_df = spark.createDataFrame(
    [(residual_stats['mean_abs_error'], 
      residual_stats['std_abs_error'],
      residual_stats['min_abs_error'],
      residual_stats['max_abs_error'])],
    ['mean_abs_error', 'std_abs_error', 'min_abs_error', 'max_abs_error']
)
residual_df.write.mode("overwrite").saveAsTable("weather_residual_statistics")

## Section 7: Temperature Range Analysis

Analyze prediction accuracy across different temperature ranges

In [ ]:
# Temperature range analysis
print("\n" + "=" * 80)
print("TEMPERATURE RANGE ANALYSIS")
print("=" * 80)

temp_range_analysis = predictions.withColumn(
    "temp_range",
    when(col("temperature") < 0, "Below 0°C")
    .when(col("temperature") < 10, "0-10°C")
    .when(col("temperature") < 20, "10-20°C")
    .when(col("temperature") < 30, "20-30°C")
    .otherwise("Above 30°C")
).groupBy("temp_range").agg(
    count("*").alias("predictions"),
    spark_round(avg("abs_error"), 4).alias("avg_error"),
    spark_round(avg("temperature"), 2).alias("avg_temp"),
    (count(when(col("abs_error") <= 2.0, 1)) / count("*") * 100).alias("accuracy_within_2c_pct")
)

print("\nPerformance by Temperature Range:")
temp_range_analysis.show(truncate=False)

# Save temperature range analysis
temp_range_analysis.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("weather_analysis_by_temp_range")

## Section 8: Summary and Final Report

Generate comprehensive summary of analysis results

In [ ]:
# Final summary report
print("\n" + "=" * 80)
print("WEATHER ML PREDICTION ANALYSIS - FINAL SUMMARY")
print("=" * 80)

print(f"\nDataset Summary:")
print(f"  Total Predictions: {predictions.count():,}")
print(f"  Mean Temperature: {predictions.select(avg('temperature')).collect()[0][0]:.2f}°C")
print(f"  Mean Error: {residual_stats['mean_abs_error']:.4f}°C")

print(f"\nAnalysis Tables Created:")
print(f"  ✓ weather_analysis_by_city")
print(f"  ✓ weather_analysis_by_hour (if hour data available)")
print(f"  ✓ weather_analysis_by_day (if day data available)")
print(f"  ✓ weather_analysis_by_condition (if condition data available)")
print(f"  ✓ weather_analysis_by_temp_range")
print(f"  ✓ weather_residual_statistics")

print(f"\nKey Findings:")
print(f"  - Best accuracy within: 2.0°C")
print(f"  - Average prediction error: {residual_stats['mean_abs_error']:.4f}°C")
print(f"  - Standard deviation: {residual_stats['std_abs_error']:.4f}°C")

print("\n" + "=" * 80)
print("PREDICTION ANALYSIS COMPLETED SUCCESSFULLY")
print("=" * 80)
print("\nNext Steps:")
print("  1. Review city-wise performance table")
print("  2. Monitor weather conditions with high errors")
print("  3. Consider retraining with more data if accuracy is low")
print("  4. Use predictions for downstream applications")
print("=" * 80)